In [1]:
import numpy
import tensorflow as tf
import types
import time
from utils import TBLogger
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import ops
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)
numpy.set_printoptions(precision=3, suppress=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
def ffnn(x, hparams):
    sizes = [hparams.n_inputs] + [hparams.n_hidden for _ in range(hparams.n_layers)] + [hparams.n_targets]
    for i in range(len(sizes) - 1):
        w = tf.Variable(tf.truncated_normal(sizes[i:i+2], stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[sizes[i+1]]))
        x = tf.matmul(x, w) + b
        
        shift = tf.reduce_mean(w, axis=1, keepdims=True)
        relative = w = tf.tile(shifts, [hparams.n_clients, 1])
        
        shifts = tf.expand_dims(tf.reduce_mean(weights, axis=0), 0)
    relative = weights - tf.tile(shifts, [hparams.n_clients, 1]) + hparams.market_shift * tf.ones([hparams.n_clients, 1 + hparams.n_experts]) 
    masks = tf.nn.relu(relative)
    
    return x


In [9]:
def model_fn(hparams):    
    inputs = tf.placeholder("float", [None, hparams.n_inputs], 'inputs')
    targets = tf.placeholder("float", [None, hparams.n_targets], 'targets')   
    
    masks = []
    x = inputs
    sizes = [hparams.n_inputs] + [hparams.n_hidden for _ in range(hparams.n_layers)] + [hparams.n_targets]
    for i in range(len(sizes) - 1):
        
        # Layer weights.
        w = tf.Variable(tf.truncated_normal(sizes[i:i+2], stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[sizes[i+1]]))
        
        # Layer flows
        f = tf.Variable(tf.truncated_normal(sizes[i:i+2], stddev=0.1))
        
        # Flow from neuron i to j sums to 1. 
        f = tf.linalg.normalize(f, axis=0, ord=1)[0]
        
        # Market shift
        s = tf.tile(tf.reduce_mean(f, axis=0, keepdims=True), [sizes[i], 1])
        relative = f - s
        mask = tf.nn.relu(relative)
        masks.append(mask)
        
        # Apply the mask to the weights.
        w = tf.multiply(w, mask)
        
        # Use the weights.
        x = tf.matmul(x, w) + b 
                
    logits = x
        
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=targets, logits=logits))
    correct = tf.equal(tf.argmax(logits, 1), tf.argmax(targets, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    full_loss = loss # + (hparams.alpha * norm_sum)
    
    train_step = tf.train.AdamOptimizer(hparams.learning_rate).minimize(full_loss)
    
    metrics = {
        'loss': loss,
        'accuracy': accuracy,
    }
    for i,m in enumerate(masks):
        metrics['sparsity_in_mask_' + str(i)] = tf.nn.zero_fraction(m)
        
    return train_step, metrics

In [ ]:
hparams = types.SimpleNamespace( 
    batch_size=256,
    learning_rate=1e-3,
    n_inputs = 784,
    n_targets = 10,
    n_layers = 2,
    n_hidden = 256,
    n_iterations = 1000000,
    n_print = 1000,
    alpha = 0.00001,
    logdir='logs/' + str(int(time.time()))
)

logger = TBLogger(hparams.logdir)
graph = tf.Graph()
session = tf.Session(graph=graph)
with graph.as_default():
    train_step, metrics = model_fn(hparams)
    session.run(tf.global_variables_initializer())

for i in range(hparams.n_iterations):
    batch_x, batch_y = mnist.train.next_batch(hparams.batch_size)
    feeds = {'inputs:0': batch_x, 'targets:0': batch_y}
    session.run(train_step, feeds)

    if i % hparams.n_print == 0:
        feeds = {'inputs:0': batch_x, 'targets:0': batch_y}
        train_metrics = session.run(metrics, feeds)
        for key in train_metrics:
            print (str(key) + ":  " + str(train_metrics[key]))
        print ('-')
            
        

loss:  2.3024254
accuracy:  0.10546875
sparsity_in_mask_0:  0.49952167
sparsity_in_mask_1:  0.49972534
sparsity_in_mask_2:  0.50078124
-
loss:  1.9941278
accuracy:  0.28125
sparsity_in_mask_0:  0.54268974
sparsity_in_mask_1:  0.6105499
sparsity_in_mask_2:  0.9515625
-
loss:  1.3186861
accuracy:  0.55078125
sparsity_in_mask_0:  0.5486488
sparsity_in_mask_1:  0.61087036
sparsity_in_mask_2:  0.9867188
-
loss:  1.1376976
accuracy:  0.5859375
sparsity_in_mask_0:  0.55245036
sparsity_in_mask_1:  0.6102295
sparsity_in_mask_2:  0.98828125
-
loss:  1.1617646
accuracy:  0.62890625
sparsity_in_mask_0:  0.55288386
sparsity_in_mask_1:  0.61161804
sparsity_in_mask_2:  0.98945314
-
loss:  0.95963544
accuracy:  0.69140625
sparsity_in_mask_0:  0.552844
sparsity_in_mask_1:  0.61172485
sparsity_in_mask_2:  0.9898438
-
loss:  0.8561187
accuracy:  0.75
sparsity_in_mask_0:  0.55279416
sparsity_in_mask_1:  0.6117401
sparsity_in_mask_2:  0.9898438
-
loss:  0.9932647
accuracy:  0.7109375
sparsity_in_mask_0:  0